In [1]:
from music21 import *
import music21
import os
import glob
import re
import numpy as np
import math

In [2]:
def demonopolize(notelist):
    if len(notelist) <=2:
        return notelist
    other_values = np.array(list(notelist.values()))
    other_values = other_values[other_values < 0.5]
    maxval = np.max(other_values)
    if len(notelist) <=2:
        return notelist
    hasChanged = True
    dontchange = []
    totalredist = 1
    for note in notelist:
        if notelist[note] >= 0.5:
            notelist[note] = maxval
    return notelist
            

def importance_score(notelist,noteduration,noteoctave):
    allnotes = {}
    for i in range(len(notelist)):
        if not notelist[i] in allnotes:
            allnotes[notelist[i]] = {"occ":1,"durlist":[noteduration[i]],"octavelist":[noteoctave[i]]}
        else:
            allnotes[notelist[i]]['occ'] += 1
            allnotes[notelist[i]]['durlist'].append(noteduration[i])
            allnotes[notelist[i]]['octavelist'].append(noteoctave[i])
    returnnote = {}
    totalscore = 0
    for note in allnotes:
        returnnote[note] = int(allnotes[note]['occ'] * (np.sum(allnotes[note]['durlist'])) * (21-2*np.min(allnotes[note]['octavelist'])))
        if returnnote[note] == 0:
            returnnote[note] = 1
        totalscore += returnnote[note]
    for note in allnotes:
        returnnote[note] = round(returnnote[note]/totalscore,3)
    retunnote = demonopolize(returnnote)
    return returnnote

In [3]:
def cal_offset(e):
    if e is None:
        return 0
    return e.offset+cal_offset(e.activeSite)

In [4]:
class Score_in_beat:
    
    def __init__(self):
        self.beat_list=[Beat()]
        
    #extract info from note and add to corrsponding beat
    def add_note(self,note):
        length=note.quarterLength
        start=cal_offset(note)
        end=start+length
        rounded_floor_start=math.floor(start)
        #loop until the note played to its end
        while start<end-0.000000000001:
            if len(self.beat_list)-1<rounded_floor_start:
                new_beat=rounded_floor_start-(len(self.beat_list)-1)
                #the input note maybe is a chord -> recurse all pitch inside
                for _ in range(new_beat):
                    self.beat_list.append(Beat())
            self.beat_list[rounded_floor_start].add_note(note,min(rounded_floor_start+1-start,end-start))
            start+=min(rounded_floor_start+1-start,end-start)
            rounded_floor_start=int(start)
            
    #add key to the first occurence of beat
    def add_key(self,note):
        assert(note.lyric is not None and '(' in note.lyric)
        key_change_beat=cal_offset(note)
        rounded_floor_key_change_beat=math.floor(key_change_beat)
        self.beat_list[rounded_floor_key_change_beat].add_key(note.lyric.split('(')[0])
        
    #onyl call once
    def infer_key(self):
        first_key_in_num=None
        first_key_full=None
        first_key_major=None
        #backtrack
        for e in self.beat_list:
            if e.key_full is not None:
                first_key_full=e.key_full
                first_key_in_num=e.key_in_num
                first_key_major=e.major
                break
        #bring forward
        for e in self.beat_list:
            if e.key_full is None:
                e.key_full=first_key_full
                e.key_in_num=first_key_in_num
                e.major=first_key_major
            else:
                first_key_full=e.key_full
                first_key_in_num=e.key_in_num
                first_key_major=e.major

In [5]:
key_mapping={
    'C':0,
    'D':2,
    'E':4,
    'F':5,
    'G':7,
    'A':9,
    'B':11
}
def key2num(k):  
    k=k.upper()
    num=key_mapping[k[0]]
    modifier=len(k)
    if modifier==1:
        return num
    elif k[1]=='#':
        return (num+(modifier-1))%12
    elif k[1]=='B' or k[1]=='-' or k[1]=='♭':
        return (num-(modifier-1))%12
    elif k[1]=='X':
        return (num+(modifier-1)*2)%12

In [6]:
class Beat:
    def __init__(self):
        self.notes = np.zeros((12,7))  #from C1 to C7
        self.total_duration = np.zeros((12,7))
        self.notes_occurences_count= np.zeros((12,7))
        self.key_full=None
        self.major=None
        self.key_in_num=None
        
    def add_note(self,note,duration):
        assert(duration<=1)
        pitches=note.pitches
        for pitch in pitches:
            pitch_idx=key2num(pitch.nameWithOctave[:-1])
            octave=int(pitch.nameWithOctave[-1])-1
            if octave<0:
                octave=0
            elif octave>6:
                octave=6
            self.notes[pitch_idx,octave]=1
            self.total_duration[pitch_idx,octave]+=duration
            self.notes_occurences_count[pitch_idx,octave]+=1
            
    def add_key(self,k):
        self.major = 'M' in k
        self.key_full=k
        k=k[:-1]
        self.key_in_num=key2num(k)

In [7]:
#piece='./Chopin_F._Nocturne_in_E_Major,_Op.26_No.2.mxl'
all_score=[]
for piece in glob.glob("../musicxml(notated)/*.mxl"):
    all_beat=Score_in_beat()
    all_score.append(all_beat)
    print(piece)
    chords = []
    notes = []
    c = converter.parse(piece)
    post = c.flat

    #extract note
    all_notes=[]
    for note in post.notes:
        all_notes.append(note)
        all_beat.add_note(note)
        if note.lyric is not None and '(' in note.lyric:
            all_beat.add_key(note)
        #print(note,note.pitches,note.pitches[0].nameWithOctave,note.quarterLength,cal_offset(note))

    all_beat.infer_key()

../musicxml(notated)\E╠ütude_in_C_Minor.mxl
../musicxml(notated)\E╠ütude_in_F_Major.mxl
../musicxml(notated)\E╠ütude_in_F_Minor.mxl
../musicxml(notated)\E╠ütude_in_Gb_Major.mxl
../musicxml(notated)\E╠ütude_in_Gb_Major_Opus_25.mxl
../musicxml(notated)\Il_Vecchio_Castello.mxl
../musicxml(notated)\Menuet_in_G_Minor.mxl
../musicxml(notated)\Minuet_in_F.mxl
../musicxml(notated)\Minuet_in_G_Major_2nd.mxl
../musicxml(notated)\Moonlight_Sonata_1st_Movement.mxl
../musicxml(notated)\Nocturne_in_B_Major.mxl
../musicxml(notated)\Nocturne_in_C#_Minor.mxl
../musicxml(notated)\Nocturne_in_Eb_Major.mxl
../musicxml(notated)\Nocturne_in_E_Minor.mxl
../musicxml(notated)\Nocturne_in_F#_Major.mxl
../musicxml(notated)\Nocturne_in_F_Minor.mxl
../musicxml(notated)\Nocturne_No._20_in_C#_Minor.mxl
../musicxml(notated)\notes-to-chord.mxl
../musicxml(notated)\Piano_Sonata_No._11.mxl
../musicxml(notated)\Pre╠ülude_in_A_Major.mxl
../musicxml(notated)\Pre╠ülude_in_B_Major.mxl
../musicxml(notated)\Pre╠ülude_in_B_Mino

In [8]:
for i in range(len(all_score)):
    print(i,all_score[i].beat_list[0].key_full)

0 Cm
1 FM
2 Fm
3 G♭M
4 G♭M
5 G#m
6 Gm
7 FM
8 GM
9 C#m
10 BM
11 C#m
12 E♭M
13 Em
14 F#M
15 Fm
16 C#m
17 E♭M
18 Am
19 AM
20 BM
21 Bm
22 Bm
23 Cm
24 D♭M
25 F#M
26 DM
27 Em
28 E♭M
29 AM
30 FM
31 Fm
32 GM
33 CM
34 A♭M
35 Am
36 E♭M


In [9]:
-----------------------------------------------------------------------------------------------------asg-----------------------

SyntaxError: invalid syntax (<ipython-input-9-4f6fca2f5360>, line 1)

In [ ]:
print(chord_sequence)
import json
with open('../data/training_data2.json','w') as f:
    d = json.dumps(chord_sequence)
    f.write(d)